In [1]:
import pandas as pd
from IPython.display import display, HTML
import numpy as np

In [2]:
# Display results in rows

##caution ! may need or not to filter out some snrs

def display_df(params, l_snr=-5,h_snr=10,colormax=0, sir_sar=False, dns_mos=False, ids_to_filter=None,id_column=None, separate=False, stat="mean"):

    if sir_sar:
        metric_columns = ["SI-SDR", "SI-SIR", "SI-SAR", "PESQ", "STOI"]

        if dns_mos:
            metric_columns += ["MOS_SIG", "MOS_BAK", "MOS_OVRL"]


    else:

        metric_columns = ["SI-SDR", "PESQ", "STOI"]

        if dns_mos:
            metric_columns  += ["MOS_SIG", "MOS_BAK", "MOS_OVRL"]
        


    if not separate:        
        combined_results = pd.DataFrame(columns=[x+ " ± SE" for x in metric_columns])        
    else:
        combined_results_mean = pd.DataFrame(columns=[x for x in metric_columns])     
        combined_results_se = pd.DataFrame(columns=[f"{x}_SE" for x in metric_columns])    

    for key, filepath in params.items():
        df = pd.read_csv(filepath)

        # Filter the DataFrame
        condition = (df['Noise SNR'] >= l_snr) & (df['Noise SNR'] <= h_snr) # & (df['Noise Type'] != 'Street') & (df['Noise Type'] != 'Cafe')
        df_filtered = df[condition]

        if ids_to_filter is not None and id_column is not None:
            df_filtered = df_filtered[df_filtered[id_column].isin(ids_to_filter)]

        # print("len df len(df_filtered) :",len(df_filtered))
        # Calculate average and standard error
        if stat=="mean":
            average = df_filtered[metric_columns].mean()
        elif stat=="median":
            average = df_filtered[metric_columns].median()

        standard_error = df_filtered[metric_columns].std() / np.sqrt(len(df_filtered))

        # Round the values to two decimal places
        average = average.round(2)
        standard_error = standard_error.round(2)
        
        if not separate:
            # Create a new DataFrame for the data frame's metrics and their standard errors
            df_row = pd.DataFrame({f"{metric} ± SE": [f"{average[metric]} ± {standard_error[metric]}"] for metric in metric_columns})
            df_row.index = [key]

            # Append the data frame's row to the combined results DataFrame
            combined_results = pd.concat([combined_results, df_row])
        
        #if name != []:
            #combined_results.index = name

        else:

            # Create a new DataFrame for the data frame's metrics and their standard errors
            df_row_mean = pd.DataFrame({f"{metric}": [average[metric]] for metric in metric_columns})
            df_row_mean.index = [key]

            # Append the data frame's row to the combined results DataFrame
            combined_results_mean = pd.concat([combined_results_mean, df_row_mean])

            df_row_se = pd.DataFrame({f"{metric}_SE": [standard_error[metric]] for metric in metric_columns})
            df_row_se.index = [key]

            # Append the data frame's row to the combined results DataFrame
            combined_results_se = pd.concat([combined_results_se, df_row_se])
                

    if not separate:
        # Display the combined results
        if colormax == 0:
            display(combined_results)
            # return combined_results
            
    
        elif colormax == 1 :
            display(combined_results.style.pipe(make_pretty_1))
        
        elif colormax == 2 :
            display(combined_results.style.pipe(make_pretty_2))

        # return combined_results
    
    else: 
        display(combined_results_mean)
        # return combined_results_mean, combined_results_se

In [3]:
def mean_per_group(dict_, group = "snr" , colormax=0) :
    
    metrics = ["SI-SDR","PESQ","STOI"]

    if group == "snr":
        a_df = pd.read_csv( dict_[list(dict_.keys())[0]] ) 
        elements_group = list(a_df.groupby(["Noise SNR"]).mean().index)
        
        
    elif group == "ntype":
        a_df = pd.read_csv( dict_[list(dict_.keys())[0]] )  
        elements_group = list(a_df.groupby(["Noise Type"]).mean().index)


    dico = {}


    for (name,file) in dict_.items() :
        
        general_df = pd.read_csv(file)

        if group == "snr":
            df = round(general_df.groupby(["Noise SNR"]).mean(),2)
        
        elif group == "ntype":
            df = round(general_df.groupby(["Noise Type"]).mean(),2)        
        
        values = []
        
        #values.extend(df.si_sar.tolist())
        #values.extend(df.si_sir)
        #values.extend(df.estoi)

        values.extend(df["SI-SDR"])
        values.extend(df["PESQ"])
        values.extend(df["STOI"])    
            
        dico[name] = values


    df_per_group = pd.DataFrame.from_dict(dico, orient='index')#,index=nickname)

    ##cartesian product
    metric_levels = [item for item in metrics for i in range(len(elements_group))]
    group_levels = elements_group*len(metrics)
    tuples = list(zip(metric_levels,group_levels))

    if group == "snr":
        df_per_group.columns = pd.MultiIndex.from_tuples(tuples, names=["Metrics", "SNRs"])
        #df_per_group.index = list(dict_.keys())
    elif group == "ntype":
        df_per_group.columns = pd.MultiIndex.from_tuples(tuples, names=["Metrics", "Noise types"])
        #df_per_group.index = list(dict_.keys())


    # return df_per_group
    if colormax == 0:
        display(df_per_group)
 
    elif colormax == 1 :
        display(df_per_group.style.pipe(make_pretty_1))
    
    elif colormax == 2 :
        display(df_per_group.style.pipe(make_pretty_2))


def make_pretty_1(styler):
    styler.format(precision=2, thousands=".", decimal=",") 
    styler.highlight_max(color = 'color:white;background-color:darkblue', axis = 0)    
    #styler.apply(highlight_max, props1='color:white;background-color:darkblue', axis=0)    
    return styler

def make_pretty_2(styler):
    styler.format(precision=2, thousands=".", decimal=",") 
    # styler.highlight_max(color = 'color:white;background-color:darkblue', axis = 0)    
    styler.apply(highlight_max, props1='color:white;background-color:darkblue', axis=0)    
    styler.apply(highlight_secondmax, props2='color:white;background-color:lightblue', axis=0)
    return styler

def highlight_max(s, props1=''):
    if s.dtype ==object:
        s = s.apply(lambda x: x.split("±")[0])
        s = s.astype(float)    
    return np.where(s == list(s.nlargest(2))[0], props1,'') 

def highlight_secondmax(s, props2=''):
    if s.dtype ==object:
        s = s.apply(lambda x: x.split("±")[0])
        s = s.astype(float)
    return np.where(s == list(s.nlargest(2))[1], props2,'')  


In [4]:
params_tcd_demand = {
"fudiffse":"/group_cacul/calcul/users/jayilo/ICP_52_results/grat/TCD-DEMAND/GTA_aonly_tcd_speech_modeling_default_28M/TCD-DEMAND/fudiffse/fudiffuse_bs4/metrics.csv",
"udiffse":"/group_cacul/calcul/users/jayilo/ICP_52_results/grat/TCD-DEMAND/GTA_aonly_tcd_speech_modeling_default_28M/TCD-DEMAND/udiffse/udiffuse/metrics.csv",
"av-fudiffse":"/group_cacul/calcul/users/jayilo/ICP_52_results/grat/TCD-DEMAND/av_tcd_speech_modeling_concat_attn_masking_light_avhubert_p0_28M_enc_dec/TCD-DEMAND/fudiffse/fudiffuse_bs4/metrics.csv",
"av-udiffse":"/group_cacul/calcul/users/jayilo/ICP_52_results/grat/TCD-DEMAND/av_tcd_speech_modeling_concat_attn_masking_light_avhubert_p0_28M_enc_dec/TCD-DEMAND/udiffse/udiffuse/metrics.csv",
# "flow_avse_stage1":"/groupe_cacul/calcul/users/jayilo/ICP_52_results/grat/TCD-DEMAND/flow_avse_enh_tcd_demand_best_sisdr/first_stage/metrics.csv",
"flow_avse_stage2":"/group_cacul/calcul/users/jayilo/ICP_52_results/grat/TCD-DEMAND/flow_avse_enh_tcd_demand_best_sisdr/second_stage/metrics.csv"
}



In [5]:
params_lrs3_ntcd = {
"fudiffse":"/group_cacul/calcul/users/jayilo/ICP_52_results/grat/LRS3-NTCD/GTA_aonly_tcd_speech_modeling_default_28M/LRS3-NTCD/fudiffse/fudiffuse_bs4/metrics.csv",
"udiffse":"/group_cacul/calcul/users/jayilo/ICP_52_results/grat/LRS3-NTCD/GTA_aonly_tcd_speech_modeling_default_28M/LRS3-NTCD/udiffse/udiffuse/metrics.csv",
"av-fudiffse":"/group_cacul/calcul/users/jayilo/ICP_52_results/grat/LRS3-NTCD/av_tcd_speech_modeling_concat_attn_masking_light_avhubert_p0_28M_enc_dec/LRS3-NTCD/fudiffse/fudiffuse/metrics.csv",
"av-udiffse": "/group_cacul/calcul/users/jayilo/ICP_52_results/grat/LRS3-NTCD/av_tcd_speech_modeling_concat_attn_masking_light_avhubert_p0_28M_enc_dec/LRS3-NTCD/udiffse/udiffuse/metrics.csv",
# "flow_avse_stage1":"/group_cacul/calcul/users/jayilo/ICP_52_results/grat/LRS3-NTCD/flow_avse_enh_lrs3_ntcd_best_sisdr/first_stage/metrics.csv",
"flow_avse_stage2":"/group_cacul/calcul/users/jayilo/ICP_52_results/grat/LRS3-NTCD/flow_avse_enh_lrs3_ntcd_best_sisdr/second_stage/metrics.csv"
}


In [6]:
display_df(params_lrs3_ntcd, l_snr=-20,h_snr=20,colormax=0, sir_sar=True, dns_mos=False, ids_to_filter=None,id_column=None, separate=False, stat="mean")


,SI-SDR ± SE,SI-SIR ± SE,SI-SAR ± SE,PESQ ± SE,STOI ± SE
fudiffse,2.95 ± 0.15,13.86 ± 0.31,3.98 ± 0.12,2.31 ± 0.02,0.59 ± 0.01
udiffse,4.72 ± 0.16,11.57 ± 0.26,6.52 ± 0.13,2.38 ± 0.02,0.62 ± 0.01
av-fudiffse,3.96 ± 0.13,21.25 ± 0.34,4.38 ± 0.11,2.46 ± 0.02,0.63 ± 0.0
av-udiffse,5.6 ± 0.15,18.37 ± 0.3,6.29 ± 0.14,2.48 ± 0.02,0.64 ± 0.01
flow_avse_stage2,3.12 ± 0.23,21.24 ± 0.32,3.28 ± 0.23,1.49 ± 0.02,0.53 ± 0.01


In [7]:
mean_per_group(params_lrs3_ntcd, group = "ntype", colormax=0) 

Metrics          SI-SDR                            PESQ                    \
Noise types      Babble  Cafe    Car    LR White Babble  Cafe   Car    LR   
fudiffse           1.62  2.30   3.69  1.46  5.60   2.01  2.05  3.19  2.31   
udiffse            2.76  3.59   7.25  3.52  6.51   1.99  2.06  3.47  2.37   
av-fudiffse        2.26  3.06   7.45  2.15  5.01   2.16  2.23  3.27  2.44   
av-udiffse         3.13  4.11  11.29  4.04  5.65   2.13  2.20  3.48  2.46   
flow_avse_stage2  -1.41  0.25   9.97  4.53  2.45   1.02  1.18  2.46  1.68   

Metrics                  STOI                          
Noise types      White Babble  Cafe   Car    LR White  
fudiffse          2.05   0.49  0.51  0.79  0.62  0.54  
udiffse           2.06   0.51  0.53  0.87  0.66  0.56  
av-fudiffse       2.25   0.54  0.55  0.82  0.65  0.59  
av-udiffse        2.19   0.54  0.56  0.88  0.67  0.58  
flow_avse_stage2  1.12   0.39  0.43  0.79  0.60  0.45

In [8]:
mean_per_group(params_lrs3_ntcd, group = "snr", colormax=0) 

Metrics          SI-SDR        PESQ        STOI      
SNRs                 -5     5    -5     5    -5     5
fudiffse          -1.30  7.15  1.92  2.70  0.46  0.71
udiffse           -0.06  9.45  1.97  2.79  0.50  0.75
av-fudiffse        1.21  6.68  2.14  2.78  0.53  0.73
av-udiffse         2.24  8.92  2.12  2.85  0.53  0.75
flow_avse_stage2  -0.99  7.19  1.15  1.82  0.41  0.65

In [9]:
display_df(params_tcd_demand, l_snr=-20,h_snr=20,colormax=0, sir_sar=True, dns_mos=False, ids_to_filter=None,id_column=None, separate=False, stat="mean")


,SI-SDR ± SE,SI-SIR ± SE,SI-SAR ± SE,PESQ ± SE,STOI ± SE
fudiffse,8.99 ± 0.21,23.98 ± 0.52,10.01 ± 0.16,3.19 ± 0.02,0.72 ± 0.01
udiffse,11.69 ± 0.27,20.74 ± 0.48,13.73 ± 0.21,3.08 ± 0.02,0.77 ± 0.01
av-fudiffse,10.21 ± 0.15,31.88 ± 0.5,10.57 ± 0.14,3.26 ± 0.02,0.74 ± 0.0
av-udiffse,13.7 ± 0.23,26.94 ± 0.48,14.64 ± 0.2,3.18 ± 0.02,0.79 ± 0.01
flow_avse_stage2,17.83 ± 0.18,39.9 ± 0.36,17.89 ± 0.18,3.18 ± 0.02,0.82 ± 0.0


In [10]:

mean_per_group(params_tcd_demand, group = "ntype", colormax=0) 

Metrics           SI-SDR                                    PESQ           \
Noise types      OOFFICE SPSQUARE STRAFFIC   TBUS TMETRO OOFFICE SPSQUARE   
fudiffse           13.56     7.61     5.72   8.90   9.16    3.59     3.04   
udiffse            19.23     9.48     6.50  12.05  11.17    3.53     2.88   
av-fudiffse        13.14     9.87     7.61  10.94   9.49    3.60     3.14   
av-udiffse         19.65    12.57     8.98  15.20  12.10    3.60     2.97   
flow_avse_stage2   22.70    15.92    14.92  20.52  15.09    3.62     2.96   

Metrics                                   STOI                                 
Noise types      STRAFFIC  TBUS TMETRO OOFFICE SPSQUARE STRAFFIC  TBUS TMETRO  
fudiffse             2.82  3.52   2.99    0.84     0.67     0.61  0.80   0.68  
udiffse              2.66  3.53   2.84    0.89     0.72     0.65  0.86   0.72  
av-fudiffse          2.96  3.54   3.07    0.84     0.71     0.66  0.81   0.69  
av-udiffse           2.79  3.58   2.97    0.90     0.75     0.70  0.87   0.74  
flow_avse_stage2     2.88  3.53   2.90    0.91     0.79     0.75  0.89   0.77

In [11]:
mean_per_group(params_tcd_demand, group = "snr", colormax=0) 

Metrics          SI-SDR         PESQ        STOI      
SNRs                 -5      5    -5     5    -5     5
fudiffse           4.93  13.05  2.90  3.48  0.62  0.81
udiffse            6.64  16.73  2.77  3.40  0.67  0.86
av-fudiffse        7.63  12.79  3.01  3.52  0.66  0.82
av-udiffse         9.94  17.46  2.88  3.48  0.71  0.87
flow_avse_stage2  14.78  20.88  2.88  3.47  0.75  0.89